In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns

# Configure display options for better readability
pd.set_option('display.max_columns', None)


In [ ]:
# Load the raw dataset from the DVC-tracked folder
df_raw = pd.read_csv('../data/raw/MachineLearningRating_v3.txt', sep='\t')

# Display the first few rows to verify loading
df_raw.head()


,UnderwrittenCoverID|PolicyID|TransactionMonth|IsVATRegistered|Citizenship|LegalType|Title|Language|Bank|AccountType|MaritalStatus|Gender|Country|Province|PostalCode|MainCrestaZone|SubCrestaZone|ItemType|mmcode|VehicleType|RegistrationYear|make|Model|Cylinders|cubiccapacity|kilowatts|bodytype|NumberOfDoors|VehicleIntroDate|CustomValueEstimate|AlarmImmobiliser|TrackingDevice|CapitalOutstanding|NewVehicle|WrittenOff|Rebuilt|Converted|CrossBorder|NumberOfVehiclesInFleet|SumInsured|TermFrequency|CalculatedPremiumPerTerm|ExcessSelected|CoverCategory|CoverType|CoverGroup|Section|Product|StatutoryClass|StatutoryRiskType|TotalPremium|TotalClaims
0,145249|12827|2015-03-01 00:00:00|True| |Close...
1,145249|12827|2015-05-01 00:00:00|True| |Close...
2,145249|12827|2015-07-01 00:00:00|True| |Close...
3,145255|12827|2015-05-01 00:00:00|True| |Close...
4,145255|12827|2015-07-01 00:00:00|True| |Close...


In [11]:
df_raw = pd.read_csv('../data/raw/MachineLearningRating_v3.txt', sep='|')


C:\Users\metya\AppData\Local\Temp\ipykernel_29436\1495679017.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv('../data/raw/MachineLearningRating_v3.txt', sep='|')


In [12]:
print(df_raw.columns.tolist())

['UnderwrittenCoverID', 'PolicyID', 'TransactionMonth', 'IsVATRegistered', 'Citizenship', 'LegalType', 'Title', 'Language', 'Bank', 'AccountType', 'MaritalStatus', 'Gender', 'Country', 'Province', 'PostalCode', 'MainCrestaZone', 'SubCrestaZone', 'ItemType', 'mmcode', 'VehicleType', 'RegistrationYear', 'make', 'Model', 'Cylinders', 'cubiccapacity', 'kilowatts', 'bodytype', 'NumberOfDoors', 'VehicleIntroDate', 'CustomValueEstimate', 'AlarmImmobiliser', 'TrackingDevice', 'CapitalOutstanding', 'NewVehicle', 'WrittenOff', 'Rebuilt', 'Converted', 'CrossBorder', 'NumberOfVehiclesInFleet', 'SumInsured', 'TermFrequency', 'CalculatedPremiumPerTerm', 'ExcessSelected', 'CoverCategory', 'CoverType', 'CoverGroup', 'Section', 'Product', 'StatutoryClass', 'StatutoryRiskType', 'TotalPremium', 'TotalClaims']


In [13]:
# Removing columns with excessive missing values (>70%)
missing_threshold = 0.7
missing_frac = df_raw.isnull().mean()
cols_to_drop = missing_frac[missing_frac > missing_threshold].index
df_clean = df_raw.drop(columns=cols_to_drop)

# Filter out invalid records (e.g., negative premiums or claims)
df_clean = df_clean[(df_clean['TotalPremium'] >= 0) & (df_clean['TotalClaims'] >= 0)]

# Converting relevant columns to categorical
cat_cols = ['Province', 'ZipCode', 'Gender', 'VehicleMake']
for col in cat_cols:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].astype('category')

# Create KPIs for hypothesis testing
# Claim Frequency: 1 if TotalClaims > 0 else 0
df_clean['ClaimFrequency'] = (df_clean['TotalClaims'] > 0).astype(int)

# Claim Severity: Average claim amount given a claim occurred
df_clean['ClaimSeverity'] = np.where(df_clean['ClaimFrequency'] == 1,
                                    df_clean['TotalClaims'] / df_clean['ClaimFrequency'],
                                    np.nan)

# Margin: TotalPremium minus TotalClaims
df_clean['Margin'] = df_clean['TotalPremium'] - df_clean['TotalClaims']

# Display cleaned data summary
df_clean.describe(include='all')


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,MaritalStatus,Gender,Country,Province,PostalCode,MainCrestaZone,SubCrestaZone,ItemType,mmcode,VehicleType,RegistrationYear,make,Model,Cylinders,cubiccapacity,kilowatts,bodytype,NumberOfDoors,VehicleIntroDate,AlarmImmobiliser,TrackingDevice,CapitalOutstanding,NewVehicle,WrittenOff,Rebuilt,Converted,SumInsured,TermFrequency,CalculatedPremiumPerTerm,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims,ClaimFrequency,ClaimSeverity,Margin
count,999805.000000,999805.000000,999805,999805,999805,999805,999805,999805,853844,959573,991546,990269,999805,999805,999805.000000,999805,999805,999805,9.992550e+05,999255,999805.000000,999255,999255,999255.000000,999255.000000,999255.000000,999255,999255.000000,999255,999805,999805,999803.0,846755,358167,358167,358167,9.998050e+05,999805,999805.000000,999805,999805,999805,999805,999805,999805,999805,999805,999805.000000,999805.000000,999805.000000,2788.000000,999805.000000
unique,NaN,NaN,23,2,4,6,5,1,11,3,3,3,1,9,NaN,16,45,1,NaN,5,NaN,46,411,NaN,NaN,NaN,13,NaN,174,2,2,1011.0,2,2,2,2,NaN,2,NaN,13,28,22,14,5,4,1,1,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,2015-08-01 00:00:00,False,,Individual,Mr,English,First National Bank,Current account,Not specified,Not specified,South Africa,Gauteng,NaN,Transvaal (all except Pretoria),Johannesburg,Mobility - Motor,NaN,Passenger Vehicle,NaN,TOYOTA,QUANTUM 2.7 SESFIKILE 16s,NaN,NaN,NaN,B/S,NaN,4/2012,Yes,No,0.0,More than 6 months,No,No,No,NaN,Monthly,NaN,No excess,Passenger Liability,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Commercial Cover: Monthly,Commercial,IFRS Constant,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,106745,994783,895159,911649,933263,999805,260806,597917,985915,940938,999805,393842,NaN,296180,176010,999805,NaN,933309,NaN,813115,186080,NaN,NaN,NaN,844124,NaN,287625,999568,656340,665878.0,845175,358135,358135,358080,NaN,999262,NaN,791004,104134,104145,823915,827916,914749,999805,999805,NaN,NaN,NaN,NaN,NaN
mean,104841.682168,7958.490981,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3020.206203,NaN,NaN,NaN,5.488213e+07,NaN,2010.225069,NaN,NaN,4.046653,2466.737076,97.206206,NaN,4.019259,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.042030e+05,NaN,117.869869,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61.953979,64.898858,0.002789,23273.387063,-2.944880
std,63278.947254,5289.143733,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2650.065397,NaN,NaN,NaN,1.359714e+07,NaN,3.261699,NaN,NaN,0.294057,442.837990,19.394873,NaN,0.468374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.508366e+06,NaN,399.741089,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,230.275389,2384.386132,0.052733,38719.512597,2367.438253
min,1.000000,14.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,4.041200e+06,NaN,1987.000000,NaN,NaN,0.000000,0.000000,0.000000,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e-02,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,139.043860,-392848.566930
25%,55164.000000,4500.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,821.000000,NaN,NaN,NaN,6.005692e+07,NaN,2008.000000,NaN,NaN,4.000000,2237.000000,75.000000,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e+03,NaN,3.223900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,1680.728070,0.000000
50%,94097.000000,7072.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000.000000,NaN,NaN,NaN,6.005842e+07,NaN,2011.000000,NaN,NaN,4.000000,2694.000000,111.000000,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.500000e+03,NaN,8.436900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.179737,0.000000,0.000000,6140.350877,2.159298
75%,139193.000000,11079.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4171.000000,NaN,NaN,NaN,6.005842e+07,NaN,2013.000000,NaN,NaN,4.000000,2694.000000,111.000000,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.500000e+05,NaN,90.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.929825,0

## Province-based Data Segmentation

In [14]:
#  Data Segmentation: Province-based segmentation
province_a = 'Gauteng'
province_b = 'Western Cape'

group_a = df_clean[df_clean['Province'] == province_a]
group_b = df_clean[df_clean['Province'] == province_b]

print(f"Group A size ({province_a}): {len(group_a)}")
print(f"Group B size ({province_b}): {len(group_b)}")


Group A size (Gauteng): 393842
Group B size (Western Cape): 170778


### Statistical Testing

In [ ]:
#Claim Frequency (Categorical) — Chi-Squared Test

# Create contingency table for claim frequency by province
contingency_table = pd.crosstab(df_clean['Province'].isin([province_a, province_b]),
                                df_clean['ClaimFrequency'])

# Perform chi-squared test
chi2, p_freq, dof, expected = chi2_contingency(contingency_table)

print(f"Chi-squared test for Claim Frequency between {province_a} and {province_b}:")
print(f"Chi2 Statistic = {chi2:.4f}, p-value = {p_freq:.4f}")


Chi-squared test for Claim Frequency between Gauteng and Western Cape:
Chi2 Statistic = 20.0416, p-value = 0.0000


In [ ]:
# Claim Severity (Continuous) — Independent t-test

# Extract claim severity values for each province (drop NaNs)
severity_a = group_a['ClaimSeverity'].dropna()
severity_b = group_b['ClaimSeverity'].dropna()

# Perform t-test
t_stat, p_severity = ttest_ind(severity_a, severity_b, equal_var=False)

print(f"T-test for Claim Severity between {province_a} and {province_b}:")
print(f"T-statistic = {t_stat:.4f}, p-value = {p_severity:.4f}")


T-test for Claim Severity between Gauteng and Western Cape:
T-statistic = -2.1685, p-value = 0.0306


In [ ]:
# Margin (Continuous) — Independent t-test

margin_a = group_a['Margin']
margin_b = group_b['Margin']

t_stat_margin, p_margin = ttest_ind(margin_a, margin_b, equal_var=False)

print(f"T-test for Margin between {province_a} and {province_b}:")
print(f"T-statistic = {t_stat_margin:.4f}, p-value = {p_margin:.4f}")


T-test for Margin between Gauteng and Western Cape:
T-statistic = -1.3950, p-value = 0.1630


In [18]:
# Count the number of policies per zip code
zip_counts = df_clean['PostalCode'].value_counts()

# Display the top 10 most frequent zip codes
print(zip_counts.head(10))



PostalCode
2000    133488
122      49171
7784     28582
299      25546
7405     18518
458      13775
8000     11794
2196     11048
470      10226
7100     10161
Name: count, dtype: int64


In [ ]:
zip_a = 2000  
zip_b = 122   

group_zip_a = df_clean[df_clean['PostalCode'] == zip_a]
group_zip_b = df_clean[df_clean['PostalCode'] == zip_b]

print(f"Group size for Zip {zip_a}: {len(group_zip_a)}")
print(f"Group size for Zip {zip_b}: {len(group_zip_b)}")


Group size for Zip 2000: 133488
Group size for Zip 122: 49171


In [ ]:
#Perform Chi-Squared Test for Claim Frequency by Zip Code

contingency_zip = pd.crosstab(df_clean['PostalCode'].isin([zip_a, zip_b]), df_clean['ClaimFrequency'])

chi2_zip, p_zip_freq, dof_zip, expected_zip = chi2_contingency(contingency_zip)

print(f"Chi-squared test for Claim Frequency between Zip {zip_a} and Zip {zip_b}:")
print(f"Chi2 Statistic = {chi2_zip:.4f}, p-value = {p_zip_freq:.4f}")


Chi-squared test for Claim Frequency between Zip 2000 and Zip 122:
Chi2 Statistic = 83.4687, p-value = 0.0000


In [25]:
# Perform t-tests for Claim Severity and Margin by Zip Code

severity_zip_a = group_zip_a['ClaimSeverity'].dropna()
severity_zip_b = group_zip_b['ClaimSeverity'].dropna()

t_stat_zip_severity, p_zip_severity = ttest_ind(severity_zip_a, severity_zip_b, equal_var=False)

print(f"T-test for Claim Severity between Zip {zip_a} and Zip {zip_b}:")
print(f"T-statistic = {t_stat_zip_severity:.4f}, p-value = {p_zip_severity:.4f}")

margin_zip_a = group_zip_a['Margin']
margin_zip_b = group_zip_b['Margin']

t_stat_zip_margin, p_zip_margin = ttest_ind(margin_zip_a, margin_zip_b, equal_var=False)

print(f"T-test for Margin between Zip {zip_a} and Zip {zip_b}:")
print(f"T-statistic = {t_stat_zip_margin:.4f}, p-value = {p_zip_margin:.4f}")


T-test for Claim Severity between Zip 2000 and Zip 122:
T-statistic = 0.3854, p-value = 0.7002
T-test for Margin between Zip 2000 and Zip 122:
T-statistic = 1.1642, p-value = 0.2443


## Hypothesis Testing for Gender Differences

In [26]:
# Define groups by Gender
group_men = df_clean[df_clean['Gender'] == 'Male']
group_women = df_clean[df_clean['Gender'] == 'Female']

print(f"Number of policies for Men: {len(group_men)}")
print(f"Number of policies for Women: {len(group_women)}")


Number of policies for Men: 42576
Number of policies for Women: 6755


### Statistical Tests

In [31]:
from scipy.stats import chi2_contingency

# Create contingency table for claim frequency by gender
contingency_gender = pd.crosstab(df_clean['Gender'], df_clean['ClaimFrequency'])

# Perform chi-squared test
chi2_gender, p_gender_freq, dof_gender, expected_gender = chi2_contingency(contingency_gender)

print(f"Chi-squared test for Claim Frequency by Gender:")
print(f"Chi2 Statistic = {chi2_gender:.4f}, p-value = {p_gender_freq:.4f}")


Chi-squared test for Claim Frequency by Gender:
Chi2 Statistic = 6.9986, p-value = 0.0302


In [32]:
from scipy.stats import ttest_ind

# Extract claim severity values for men and women (drop missing values)
severity_men = group_men['ClaimSeverity'].dropna()
severity_women = group_women['ClaimSeverity'].dropna()

# Perform independent t-test (Welch’s t-test)
t_stat_gender, p_gender_severity = ttest_ind(severity_men, severity_women, equal_var=False)

print(f"T-test for Claim Severity by Gender:")
print(f"T-statistic = {t_stat_gender:.4f}, p-value = {p_gender_severity:.4f}")


T-test for Claim Severity by Gender:
T-statistic = -0.5790, p-value = 0.5680


In [33]:
# Extract margin values for men and women
margin_men = group_men['Margin']
margin_women = group_women['Margin']

# Perform independent t-test
t_stat_margin_gender, p_margin_gender = ttest_ind(margin_men, margin_women, equal_var=False)

print(f"T-test for Margin by Gender:")
print(f"T-statistic = {t_stat_margin_gender:.4f}, p-value = {p_margin_gender:.4f}")


T-test for Margin by Gender:
T-statistic = -0.2077, p-value = 0.8354


In [34]:
df_clean.to_csv('../data/processed/cleaned _ task3_MachineLearningRating.csv', index=False)
